In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

import tensorflow as tf

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

##### attempt to balance

shuffled_m = np.arange(targets_all.shape[0])
np.random.shuffle(shuffled_m)

inputs_all = inputs_all[shuffled_m]
targets_all = targets_all[shuffled_m]

#num_one_targets = int(np.sum(targets_all))
num_one_targets = 400

zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
inputs = np.delete(inputs_all, indices_to_remove, axis=0)
targets = np.delete(targets_all, indices_to_remove, axis=0)

## Pre-processing

In [16]:
pd.set_option('display.max_columns', None)

raw_data = pd.read_csv('preprocessed_train.csv')
data = raw_data.drop(['PassengerId', 'Deck', 'Embarked'], axis=1)
for x in data: 
    data[x] = data[x].astype('category')

In [17]:
targets = data['Survived']
inputs = data.drop(['Survived'], axis=1)
inputs = pd.get_dummies(inputs, drop_first=False) #use all k - deal with later with weight decay 
inputs = inputs.drop(['Sex_male'], 1)

In [18]:
inputs = np.array(inputs, dtype='float32')
targets = np.array(targets, dtype='int')

In [19]:
#separate out 15% data as final test - do not touch until you're sure you're done
X, X_final, Y, y_final = train_test_split(inputs, targets, train_size = .85, random_state=33)
#separet out 15% as CV
X_train, X_cv, y_train, y_cv = train_test_split(X, Y, train_size = .85, random_state=33)

## Tensorflow

#### Base Model

In [27]:
batch_size = 32
max_epochs = 200

model = Sequential([
        Dense(10, input_dim = inputs.shape[1], activation='relu'),
        Dense(1, activation='sigmoid')])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

validation_split = 0.15
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(X_train, y_train,
          batch_size = batch_size,
          epochs = max_epochs,
          validation_split = validation_split,
          callbacks=[early_stopping],
          verbose=2)

Train on 546 samples, validate on 97 samples
Epoch 1/200
546/546 - 1s - loss: 0.7015 - accuracy: 0.4963 - val_loss: 0.6607 - val_accuracy: 0.5670
Epoch 2/200
546/546 - 0s - loss: 0.6686 - accuracy: 0.5476 - val_loss: 0.6360 - val_accuracy: 0.7216
Epoch 3/200
546/546 - 0s - loss: 0.6411 - accuracy: 0.6648 - val_loss: 0.6141 - val_accuracy: 0.7423
Epoch 4/200
546/546 - 0s - loss: 0.6167 - accuracy: 0.7216 - val_loss: 0.5936 - val_accuracy: 0.7423
Epoch 5/200
546/546 - 0s - loss: 0.5954 - accuracy: 0.7509 - val_loss: 0.5770 - val_accuracy: 0.7732
Epoch 6/200
546/546 - 0s - loss: 0.5741 - accuracy: 0.7784 - val_loss: 0.5633 - val_accuracy: 0.7938
Epoch 7/200
546/546 - 0s - loss: 0.5547 - accuracy: 0.7894 - val_loss: 0.5486 - val_accuracy: 0.7835
Epoch 8/200
546/546 - 0s - loss: 0.5348 - accuracy: 0.7985 - val_loss: 0.5334 - val_accuracy: 0.7835
Epoch 9/200
546/546 - 0s - loss: 0.5152 - accuracy: 0.8077 - val_loss: 0.5203 - val_accuracy: 0.7835
Epoch 10/200
546/546 - 0s - loss: 0.4964 - acc

In [28]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
cv_loss, cv_accuracy = model.evaluate(X_cv, y_cv, verbose=0)

print('Train loss: {0:.2f}. Train accuracy: {1:.2f}%'.format(train_loss, train_accuracy*100))
print('CV loss: {0:.2f}. CV accuracy: {1:.2f}%'.format(cv_loss, cv_accuracy*100))

Train loss: 0.39. Train accuracy: 85.69%
CV loss: 0.43. CV accuracy: 81.58%


In [29]:
pred = model.predict(X_cv) 
pred = [0 if x <0.5 else 1 for x in pred]
print(classification_report(y_cv, pred, digits=3))

              precision    recall  f1-score   support

           0      0.775     0.917     0.840        60
           1      0.884     0.704     0.784        54

    accuracy                          0.816       114
   macro avg      0.829     0.810     0.812       114
weighted avg      0.826     0.816     0.813       114



#### Param tuning

##### Optimizer = ADAM

In [93]:
def create_model(optimizer = 'adam'):
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=inputs.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [99]:
model = KerasClassifier(build_fn=create_model, 
                        verbose=0, epochs=30, batch_size=32)

In [100]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer = optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X, Y)

In [101]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.790576 using {'optimizer': 'Adam'}
0.773848 (0.021496) with: {'optimizer': 'SGD'}
0.788756 (0.021450) with: {'optimizer': 'RMSprop'}
0.678457 (0.047148) with: {'optimizer': 'Adagrad'}
0.542171 (0.054185) with: {'optimizer': 'Adadelta'}
0.790576 (0.030966) with: {'optimizer': 'Adam'}
0.779466 (0.016994) with: {'optimizer': 'Adamax'}
0.781244 (0.024752) with: {'optimizer': 'Nadam'}


##### WEIGHT INIT - 'lecun_uniform'

In [43]:
def create_model(init_mode = 'uniform'):
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=inputs.shape[1], 
                    kernel_initializer=init_mode, 
                    activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, 
                    activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [44]:
model = KerasClassifier(build_fn=create_model, 
                        verbose=0, epochs=30, batch_size=32)

In [45]:
init_mode = ['lecun_uniform', 'glorot_normal', 'glorot_uniform', 'he_uniform']
#init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

param_grid = dict(init_mode = init_mode)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X, Y)

In [46]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
#Best: 0.853337 using {'init_mode': 'he_uniform'}

Best: 0.849355 using {'init_mode': 'lecun_uniform'}
0.849355 (0.021776) with: {'init_mode': 'lecun_uniform'}
0.840119 (0.021736) with: {'init_mode': 'glorot_normal'}
0.846697 (0.023796) with: {'init_mode': 'glorot_uniform'}
0.848066 (0.018774) with: {'init_mode': 'he_uniform'}


##### activation function - 'relu'

In [47]:
def create_model(activation = 'relu'):
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=inputs.shape[1], 
                    kernel_initializer='lecun_uniform', 
                    activation=activation))
    model.add(Dense(1, kernel_initializer='lecun_uniform', 
                    activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [48]:
model = KerasClassifier(build_fn=create_model, 
                        verbose=0, epochs=30, batch_size=32)

In [50]:
activation = ['relu', 'tanh', 'sigmoid']
#activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

param_grid = dict(activation=activation)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X, Y)

In [51]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.840084 using {'activation': 'relu'}
0.840084 (0.030349) with: {'activation': 'relu'}
0.838759 (0.027375) with: {'activation': 'tanh'}
0.821593 (0.023776) with: {'activation': 'sigmoid'}


##### Drop-out - 0.2, max_norm(4)

In [59]:
import tensorflow.keras

In [53]:
from tensorflow.keras.constraints import max_norm

In [69]:
def create_model(dropout_rate = 0, weight_constraint = 0):
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=inputs.shape[1], 
                    kernel_initializer='lecun_uniform', 
                    kernel_constraint = max_norm(weight_constraint),
                    activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='lecun_uniform', 
                    activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [70]:
model = KerasClassifier(build_fn=create_model, 
                        verbose=0, epochs=30, batch_size=32)

In [71]:
weight_constraint = [1, 3, 5]
dropout_rate = [0.0, 0.2, 0.4]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=4)
grid_result = grid.fit(X, Y)

In [72]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.848079 using {'dropout_rate': 0.0, 'weight_constraint': 1}
0.848079 (0.017345) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.844110 (0.017034) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.848079 (0.010242) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.842781 (0.017854) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.842781 (0.013885) with: {'dropout_rate': 0.2, 'weight_constraint': 3}
0.838819 (0.017094) with: {'dropout_rate': 0.2, 'weight_constraint': 5}
0.840121 (0.026474) with: {'dropout_rate': 0.4, 'weight_constraint': 1}
0.842774 (0.017506) with: {'dropout_rate': 0.4, 'weight_constraint': 3}
0.833514 (0.019534) with: {'dropout_rate': 0.4, 'weight_constraint': 5}


#### Neurons - adding another layer did not outperform

In [146]:
def create_model(neurons1 = 1, neurons2 = 1):
    # create model
    model = Sequential()
    model.add(Dense(neurons1, input_dim=inputs.shape[1], 
                    kernel_initializer='lecun_uniform', 
                    kernel_constraint = max_norm(4),
                    activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(neurons2, input_dim=inputs.shape[1], 
                    kernel_initializer='lecun_uniform', 
                    kernel_constraint = max_norm(4),
                    activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='lecun_uniform', 
                    activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [147]:
model = KerasClassifier(build_fn=create_model, 
                        verbose=0, epochs=100, batch_size=32)

In [148]:
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons1=neurons, neurons2=neurons)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [149]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.836508 using {'neurons1': 10, 'neurons2': 30}
0.636508 (0.095423) with: {'neurons1': 1, 'neurons2': 1}
0.807937 (0.028659) with: {'neurons1': 1, 'neurons2': 5}
0.809524 (0.010287) with: {'neurons1': 1, 'neurons2': 10}
0.806349 (0.013654) with: {'neurons1': 1, 'neurons2': 15}
0.801587 (0.018374) with: {'neurons1': 1, 'neurons2': 20}
0.806349 (0.011878) with: {'neurons1': 1, 'neurons2': 25}
0.814286 (0.031105) with: {'neurons1': 1, 'neurons2': 30}
0.815873 (0.011224) with: {'neurons1': 5, 'neurons2': 1}
0.819048 (0.023328) with: {'neurons1': 5, 'neurons2': 5}
0.815873 (0.016187) with: {'neurons1': 5, 'neurons2': 10}
0.826984 (0.005939) with: {'neurons1': 5, 'neurons2': 15}
0.822222 (0.011878) with: {'neurons1': 5, 'neurons2': 20}
0.809524 (0.003888) with: {'neurons1': 5, 'neurons2': 25}
0.817460 (0.023757) with: {'neurons1': 5, 'neurons2': 30}
0.796825 (0.022448) with: {'neurons1': 10, 'neurons2': 1}
0.828571 (0.003888) with: {'neurons1': 10, 'neurons2': 5}
0.825397 (0.009785) wi

##### neurons w/ 1 layer -- 25

In [65]:
def create_model(neurons = 1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=inputs.shape[1], 
                    kernel_initializer='lecun_uniform', 
                    kernel_constraint = max_norm(5),
                    activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='lecun_uniform', 
                    activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [66]:
model = KerasClassifier(build_fn=create_model, 
                        verbose=0, epochs=30, batch_size=32)

In [67]:
neurons = [10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=4)
grid_result = grid.fit(X, Y)

In [68]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.849394 using {'neurons': 30}
0.830883 (0.014776) with: {'neurons': 10}
0.844117 (0.014288) with: {'neurons': 15}
0.845440 (0.015616) with: {'neurons': 20}
0.844110 (0.015753) with: {'neurons': 25}
0.849394 (0.014354) with: {'neurons': 30}


##### Batch size & epochs

In [73]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=inputs.shape[1], 
                    kernel_initializer='lecun_uniform', 
                    kernel_constraint = max_norm(5),
                    activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='lecun_uniform', 
                    activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [74]:
model =  KerasClassifier(build_fn=create_model, epochs = 30,
                           verbose=0)

In [76]:
batch_size = [40, 60, 80, 100]
epochs = [100, 150, 200]
param_grid = dict(batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                    n_jobs=-1, cv=4)

grid_result = grid.fit(X, Y)

In [77]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.849373 using {'batch_size': 60}
0.840135 (0.025054) with: {'batch_size': 40}
0.849373 (0.017175) with: {'batch_size': 60}
0.826901 (0.022230) with: {'batch_size': 80}
0.824255 (0.024922) with: {'batch_size': 100}


##### Learning rate

In [200]:
from tensorflow.keras.optimizers import SGD

In [204]:
def create_model(learn_rate = 0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(25, input_dim=inputs.shape[1], 
                    kernel_initializer='lecun_uniform', 
                    kernel_constraint = max_norm(4),
                    activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='lecun_uniform', 
                    activation='sigmoid'))
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    return model

In [205]:
model = KerasClassifier(build_fn=create_model, 
                        verbose=0, epochs=100, batch_size=60)

In [206]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [207]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.836508 using {'learn_rate': 0.2, 'momentum': 0.0}
0.715873 (0.035917) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.693651 (0.072253) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.766667 (0.029354) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.760317 (0.008094) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.787302 (0.004490) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.787302 (0.008094) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.787302 (0.014720) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.800000 (0.014019) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.804762 (0.006734) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.817460 (0.026466) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.830159 (0.022782) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.831746 (0.009785) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.830159 (0.012498) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.828571 (0.010287) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.830159 (0.008094) with: {'learn_rate': 0

#### Eval

In [87]:
batch_size = 60
max_epochs = 200

model = Sequential()
model.add(Dense(30, input_dim=inputs.shape[1], 
                kernel_initializer='lecun_uniform', 
                kernel_constraint = max_norm(5),
                activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='lecun_uniform', 
                activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
validation_split = 0.15
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(X, Y,
          batch_size = batch_size,
          epochs = max_epochs,
          validation_split = validation_split,
          callbacks=[early_stopping],
          verbose=2)

Train on 643 samples, validate on 114 samples
Epoch 1/200
643/643 - 2s - loss: 0.6625 - accuracy: 0.6205 - val_loss: 0.6160 - val_accuracy: 0.7105
Epoch 2/200
643/643 - 0s - loss: 0.6233 - accuracy: 0.6423 - val_loss: 0.5888 - val_accuracy: 0.7281
Epoch 3/200
643/643 - 0s - loss: 0.5941 - accuracy: 0.6703 - val_loss: 0.5658 - val_accuracy: 0.7456
Epoch 4/200
643/643 - 0s - loss: 0.5665 - accuracy: 0.7278 - val_loss: 0.5479 - val_accuracy: 0.7982
Epoch 5/200
643/643 - 0s - loss: 0.5423 - accuracy: 0.7574 - val_loss: 0.5333 - val_accuracy: 0.8070
Epoch 6/200
643/643 - 0s - loss: 0.5206 - accuracy: 0.7885 - val_loss: 0.5200 - val_accuracy: 0.8070
Epoch 7/200
643/643 - 0s - loss: 0.5010 - accuracy: 0.7947 - val_loss: 0.5082 - val_accuracy: 0.8070
Epoch 8/200
643/643 - 0s - loss: 0.4828 - accuracy: 0.7994 - val_loss: 0.4978 - val_accuracy: 0.8070
Epoch 9/200
643/643 - 0s - loss: 0.4667 - accuracy: 0.8196 - val_loss: 0.4897 - val_accuracy: 0.8070
Epoch 10/200
643/643 - 0s - loss: 0.4513 - ac

Epoch 81/200
643/643 - 0s - loss: 0.3104 - accuracy: 0.8818 - val_loss: 0.3664 - val_accuracy: 0.8333
Epoch 82/200
643/643 - 0s - loss: 0.3097 - accuracy: 0.8849 - val_loss: 0.3663 - val_accuracy: 0.8421
Epoch 83/200
643/643 - 0s - loss: 0.3093 - accuracy: 0.8849 - val_loss: 0.3643 - val_accuracy: 0.8421
Epoch 84/200
643/643 - 0s - loss: 0.3087 - accuracy: 0.8849 - val_loss: 0.3643 - val_accuracy: 0.8421
Epoch 85/200
643/643 - 0s - loss: 0.3079 - accuracy: 0.8849 - val_loss: 0.3639 - val_accuracy: 0.8421
Epoch 86/200
643/643 - 0s - loss: 0.3073 - accuracy: 0.8849 - val_loss: 0.3626 - val_accuracy: 0.8421
Epoch 87/200
643/643 - 0s - loss: 0.3069 - accuracy: 0.8849 - val_loss: 0.3620 - val_accuracy: 0.8421
Epoch 88/200
643/643 - 0s - loss: 0.3065 - accuracy: 0.8834 - val_loss: 0.3624 - val_accuracy: 0.8421
Epoch 89/200
643/643 - 0s - loss: 0.3058 - accuracy: 0.8849 - val_loss: 0.3615 - val_accuracy: 0.8421
Epoch 90/200
643/643 - 0s - loss: 0.3060 - accuracy: 0.8865 - val_loss: 0.3607 - v

In [84]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
cv_loss, cv_accuracy = model.evaluate(X_cv, y_cv, verbose=0)

In [85]:
print('Train loss: {0:.2f}. Train accuracy: {1:.2f}%'.format(train_loss, train_accuracy*100))
print('CV loss: {0:.2f}. CV accuracy: {1:.2f}%'.format(cv_loss, cv_accuracy*100))

Train loss: 0.36. Train accuracy: 86.63%
CV loss: 0.41. CV accuracy: 79.82%


In [86]:
print('Train loss: {0:.2f}. Train accuracy: {1:.2f}%'.format(train_loss, train_accuracy*100))
print('CV loss: {0:.2f}. CV accuracy: {1:.2f}%'.format(cv_loss, cv_accuracy*100))

Train loss: 0.36. Train accuracy: 86.63%
CV loss: 0.41. CV accuracy: 79.82%


In [88]:
cv_loss, cv_accuracy = model.evaluate(X_final, y_final, verbose=0)
print('CV loss: {0:.2f}. CV accuracy: {1:.2f}%'.format(cv_loss, cv_accuracy*100))

CV loss: 0.41. CV accuracy: 82.84%


In [90]:
pred = model.predict(X_final) 
pred = [0 if x <0.5 else 1 for x in pred]
print(classification_report(y_final, pred, digits=3))

              precision    recall  f1-score   support

           0      0.835     0.887     0.861        80
           1      0.816     0.741     0.777        54

    accuracy                          0.828       134
   macro avg      0.826     0.814     0.819       134
weighted avg      0.828     0.828     0.827       134



In [ ]:
predfinal['PassengerId'] = ids.values
predfinal = predfinal[['PassengerId', 'Survived']]
predfinal
predfinal.to_csv('nn-all-cat v6.csv', index = False)

In [229]:
submit = pd.DataFrame(raw_data['PassengerId'], columns=["PassengerId"])
submit['Survived'] = pred
submit.to_csv('nn-final.csv', index = False)

batch_size = 32
max_epochs = 200
validation_split = 0.15
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model = Sequential([
        Dense(20, input_dim = inputs.shape[1], 
              kernel_initializer='he_uniform',
              kernel_regularizer=tf.keras.regularizers.l2(0.05), 
              activation='relu'),
        Dropout(0.35),
        Dense(10, 
              kernel_initializer='he_uniform',
              kernel_regularizer=tf.keras.regularizers.l2(0.05), 
              activation='relu'),
        Dropout(0.35),
        Dense(1, activation='sigmoid')
    ])
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(inputs, 
          targets,
          batch_size = batch_size,
          epochs = max_epochs,
          validation_split = validation_split,
          callbacks=[early_stopping],
          verbose=2)


n=28
k=1
hidden_n = 20

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_n, input_dim =n, activation='sigmoid', 
                         kernel_regularizer=tf.keras.regularizers.l1(.01)),
    tf.keras.layers.Dense(hidden_n, activation='sigmoid', 
                         kernel_regularizer=tf.keras.regularizers.l1(.01)),
    tf.keras.layers.Dense(k, activation='sigmoid', 
                         kernel_regularizer=tf.keras.regularizers.l1(.01))
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

#### evaluation

In [ ]:
X, x_test, Y, y_test = train_test_split(inputs, targets, train_size = .9, random_state=82)

In [ ]:
batch_size = 32
max_epochs = 200
validation_split = 0.20
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

cvscores = []
trainscores = []

kf = KFold(n_splits=5, random_state=80, shuffle=True)

for train, test in kf.split(X, Y):
    model = Sequential([
        Dense(20, input_dim = X.shape[1], 
              kernel_initializer='he_uniform',
              kernel_regularizer=tf.keras.regularizers.l2(0.01), 
              activation='relu'),
        Dropout(0.50),
        Dense(10, 
              kernel_initializer='he_uniform',
              kernel_regularizer=tf.keras.regularizers.l2(0.01), 
              activation='relu'),
        Dropout(0.50),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    model.fit(X[train], 
              Y[train],
              batch_size = batch_size,
              epochs = max_epochs,
              validation_split = validation_split,
              callbacks=[early_stopping],
              verbose=0)
    scores_t = model.evaluate(X[train], Y[train], verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    
    #test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    #print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))
    
    print("accuracy train: %.2f%%" % (scores_t[1]*100))
    print("accuracy CV: %.2f%%" % (scores[1]*100))
    trainscores.append(scores_t[1]*100)
    cvscores.append(scores[1]*100)
print("%.2f%%(+/-%.2f%%)" % (np.mean(trainscores), np.std(trainscores)))
print("%.2f%%(+/-%.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


#return (np.mean(train_scores), np.mean(cv_scores))

In [ ]:
#03 03 ---- 01 reg
print("%.2f%%(+/-%.2f%%)" % (np.mean(trainscores), np.std(trainscores)))
print("%.2f%%(+/-%.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
#.4 .4 --- l2 0.005
print("%.2f%%(+/-%.2f%%)" % (np.mean(trainscores), np.std(trainscores)))
print("%.2f%%(+/-%.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
#.25 .25 --- 0.005 -- 80batch 

print("%.2f%%(+/-%.2f%%)" % (np.mean(trainscores), np.std(trainscores)))
print("%.2f%%(+/-%.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
print("%.2f%%(+/-%.2f%%)" % (np.mean(trainscores), np.std(trainscores)))
print("%.2f%%(+/-%.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
def test_batch(input_batch):
    batch_size = input_batch
    max_epochs = 200
    validation_split = 0.2
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=33)

    cvscores = []

    kf = KFold(n_splits=5, random_state=3, shuffle=True)

    for train, test in kf.split(inputs, targets):
        model = Sequential([
            Dense(20, input_dim = inputs.shape[1], kernel_regularizer=tf.keras.regularizers.l2(0.005), activation='relu'),
            Dropout(0.25),
            Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.005), activation='relu'),
            Dropout(0.25),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])

        model.fit(inputs[train], 
                  targets[train],
                  batch_size = batch_size,
                  epochs = max_epochs,
                  validation_split = validation_split,
                  callbacks=[early_stopping],
                  verbose=0)
        scores = model.evaluate(inputs[test], targets[test], verbose=0)

   #     print("accuracy: %.2f%%" % (scores[1]*100))
        cvscores.append(scores[1]*100)
        test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

    #print("%.2f%%(+/-%.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    return np.mean(cvscores), np.std(cvscores)

    #return (np.mean(train_scores), np.mean(cv_scores))

In [ ]:
batches = [60,80,100, 120,150]
for b in batches:
    score = test_batch(b)
    print("%.2f%%(+/-%.2f%%)" % (score[0], score[1]))

In [ ]:
prednn = model.predict(np.array(inputs)) 

predfinal = pd.DataFrame([0 if x <0.5 else 1 for x in prednn], columns = ['Survived'])
predfinal['PassengerId'] = ids.values
predfinal = predfinal[['PassengerId', 'Survived']]
predfinal

In [ ]:
predfinal['PassengerId'] = ids.values
predfinal = predfinal[['PassengerId', 'Survived']]
predfinal
predfinal.to_csv('nn-all-cat v6.csv', index = False)

### submission

In [ ]:
submit = pd.concat([my_id, pred], axis=1)
submit = submit.rename({'pred': 'Survived'}, axis=1)
submit.to_csv('log_submit_2.csv', index = False)